First, import nesassary libraries:

In [1]:
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import math
from IPython.display import HTML

The separation energy is the minimum energy needed to remove a particle from a nucleus. In our model, we will define our own seperation energy as $S_n = S_0+\sum S_k(A_0-A)^k$. In the follow calculation, we will fix the atomic number Z, so $A-A_0$ will become $N_0-N$. if k is up to 2. So our separation energy will be $S_n = S_0 + S_1(N_0-N) + S_2(N_0-N)^2$. We can define our $S_n$ as below. Where x is the array $S_n$'s, while N is the neutron number.

In reality, we also need to add the pairing and shell effects. For pair effect: when N is even, pair effect will add a constant value (delta) to x, otherwise deduct a delta. Similarly for shell effect: when N is larger than the magic number (28, 50, 82 ...), the total sn function will decrease by a constant, which we can input its value to the *magic* array. 

In [2]:
def sn_main(x, N, s_array, delta, magic):
    for i in range(len(s_array)):
        x += s_array[i] * np.power(N[0] - N, i)
        
    for i in range(len(x)):
        if (N[i] % 2) == 0:
            x[i] += delta
        else:
            x[i] -= delta
            
    for i in range(len(magic)):
        x -= magic[i][0] * np.heaviside(N - magic[i][1], 0)

    return x

When we have the separation function, we can integrate it to get the (ng)-(gn) equilibrium abundance ratio. So we will integrate $S_n$ over neutron number $N$, the useful results in the abundance equation are shown below.

In [3]:
def integral(N, delta, magic):    
    Fp = []
    shell = np.zeros(len(N))
    
    for n in range (len(N)):
        fp = delta*(-1)**(N[0]+1)*Pi(n-N[0])
        Fp.append(fp)
    Pair1 = np.array(Fp)
    
    for i in range (len(magic)):
        shell += magic[i][0] * np.heaviside(N - magic[i][1], 0)
        
    intgl = np.vstack((Pair1, shell)) 
    return intgl

def Pi(x):
    if (x % 2) == 0:
        return(0)
    else:
        return(1)

In order to get a movie, let's first create a figure with labels.

In [4]:
fig = plt.figure(figsize=(8,6))

ax1 = plt.subplot(211)

ax1.set_ylabel(r'Energy [MeV]')
ax1.set_xlim([40,110])
ax1.set_ylim([0,22])
txt_title = ax1.set_title('',fontsize=10)
line1, = ax1.plot([], [], label = r'-$\mu_n^{\prime}$')
line2, = ax1.plot([], [], label = r'$S_n$')
ax1.legend()

ax2 = plt.subplot(212)

ax2.set_ylabel(r'Abunds ratio')
ax2.set_xlabel('Neutron number N')
ax2.set_xlim([40,110])
ax2.set_ylim([1.e-40,1.e2])
ax2.set_yscale('log')
line3, = ax2.plot([], [], label = r'Abunds ratio')
plt.close()

Now, it's time to define a routine to draw a frame. 

In [5]:
def drawframe(i, N, s_array, delta, magic, t_9, Mu_n):
    T_9 = t_9[i]
    mu_n = -Mu_n[i]*N**0
    result = np.zeros(len(N))
    sn = sn_main(result, N, s_array, delta, magic)
    intl= integral(N, delta, magic) 
    txt_title.set_text(r"$T_9$ = {T_9} [MeV/k] ;  $-\mu_n'$ = {mu_n} [MeV]".format(T_9 = "%.1f" %T_9, mu_n = Mu_n[i]))
    
    f = -((s_array[1]*11.609)/(2*T_9))*((N-N[0])-(s_array[0] - intl[1,:] + mu_n)/s_array[1]+1/2)**2
    f_0 =-((s_array[1]*11.609)/(2*T_9))
    f_pair = 11.609*(intl[0,:]/T_9)
    F = math.e**(f-f_0+f_pair)
    
    line1.set_data(N,-mu_n)
    line2.set_data(N,sn)
    line3.set_data(N,F)
    
    return (line1, line2, line3)

Now, let's input the neutron number N, s_array, delta and magic numbers. Another two important indexs are the temprature($T_9$ [MeV/k]) and chemical potential $\mu_n'$ [MeV], which will change the plot significantly. Simply speaking, if $T_9$ is fixed, when the $\mu_n'$ value is equal to $S_n$, the intersection of these two curves is the location of the abundance peak. On the other hand, if the value of $\mu_n'$ is fixed, with a decreasing $T_9$, you will see a narrowing Gaussian distribution. Of course, we can change both parametes at the same time.

In [6]:
N = np.arange(40,110)
s_array = [20, 0.25, 0.001]
delta = 1.5
magic = [(3, 50), (3, 82)]
t_9 = 2*np.ones(20)  #use t_9 = np.linspace(2,0.1,20) to get a decreasing T
Mu_n = np.arange(0,20) #use Mu_n = 12*np.ones(20) to get a fixed mu_n

Now, create the movie.

In [7]:
args = (N, s_array, delta, magic, t_9, Mu_n)
anim = animation.FuncAnimation(fig, drawframe, fargs = args, frames=len(t_9), blit=True, repeat=False)
HTML(anim.to_jshtml())